In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE, mutual_info_regression, RFECV

import warnings
warnings.simplefilter(action='ignore')

# Import Data

In [ ]:
df = pd.read_csv('../input/seoul-bike-rental-ai-pro-iti/train.csv')
df.columns = [i.split("(")[0].strip().title().replace(" ","_") for i in [*df.columns]]
df.head()

# EDA

In [ ]:
df.info()

In [ ]:
data = df.copy()
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data.Date.dt.month
data['Dayofweek'] = data.Date.dt.dayofweek
data['Year'] = data.Date.dt.year
data['workend'] = (data['Dayofweek'] == 5) | (data['Dayofweek']==6)

data.set_index('Date')

data['Functioning_Day'] = data['Functioning_Day'].map({'No': 0, 'Yes': 1})
data['Holiday'] = data['Holiday'].map({'No Holiday': 0, 'Holiday': 1})
data['Year'] = data['Year'].map({2017:0 , 2018: 1})
data['Seasons'] = data['Seasons'].map({'Winter': 0, 'Spring': 1, 'Autumn': 2, 'Summer': 3})

num_cols = ['Temperature', 'Humidity', 'Wind_Speed', 'Dew_Point_Temperature', 'Rainfall', 'Snowfall', 
            'Solar_Radiation', 'Visibility']
cat_cols = ['Seasons', 'Holiday', 'Functioning_Day', 'Hour', 'Month', 'Dayofweek', 'Year', 'workend']

## Value Counts

In [ ]:
def plot_subplots(subplots, plot, n=3):
    m = len(subplots)
    height = (m//n + (m%n != 0)) * 4
    plt.figure(figsize=(14, height))
    for i, c in enumerate(subplots):
        plt.subplot(m//n + min(1, m%n), n, i+1)
        plot(c)
        plt.tight_layout(pad=2.0)

plot_subplots(num_cols + ['Y'], lambda c: sns.histplot(data=data, x=c))

In [ ]:
def plot(c):
    plt.title(c)
    data[c].value_counts().sort_index().plot.bar()
    
plot_subplots(cat_cols, lambda c: plot(c))

## Mutual Information

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

mi_scores = make_mi_scores(data.drop(columns=['Id', 'Date', 'Y']), data.Y, discrete_features=False)
plot_mi_scores(mi_scores)

## Numerical features

In [ ]:
sns.heatmap(data[num_cols + ['Y']].corr(), annot=True, cmap="YlGnBu")

In [ ]:
plot_subplots(num_cols, lambda c: sns.scatterplot(data=data, x=c, alpha=0.1, y='Y'))

### Temperature

In [ ]:
plt.figure(figsize=(20,8))
data['Temperature'].plot()
plt.show()
sns.lineplot('Month', 'Temperature', hue='Seasons', data=data)

In [ ]:
sns.lineplot('Temperature', 'Y', data=data)

In [ ]:
sns.boxplot(data['Temperature'].apply(lambda i : i >= 15 and i <=35), data['Y'])
plt.show()
sns.histplot(data['Temperature'].apply(lambda i : i >= 15 and i <=35))

### Humidity

In [ ]:
sns.lineplot(data.groupby(data.index)['Humidity'].mean(), data.groupby(data.index)['Y'].sum())

In [ ]:
sns.boxplot(data['Humidity'].apply(lambda i : i >= 20 and i <=80), data['Y'])
plt.show()
sns.histplot(data['Humidity'].apply(lambda i : i >= 20 and i <=80))

### Wind Speed

In [ ]:
sns.lineplot(data.groupby(data.index)['Wind_Speed'].mean(), data.groupby(data.index)['Y'].sum())

In [ ]:
sns.boxplot(data['Wind_Speed'].apply(lambda i : i >= 1.5 and i <=3.5), data['Y'])
plt.show()
sns.histplot(data['Wind_Speed'].apply(lambda i : i >= 1.5 and i <=3.5))

### Rainfall

In [ ]:
sns.boxplot(data['Rainfall'].apply(lambda i : i == 0), data['Y'])
plt.show()
sns.histplot(data['Rainfall'].apply(lambda i : i == 0))

### Snowfall

In [ ]:
sns.boxplot(data['Snowfall'].apply(lambda i : i == 0), data['Y'])
plt.show()
sns.histplot(data['Snowfall'].apply(lambda i : i == 0))

In [ ]:
def weather_is_good(df): 
    X = df.copy()
    X['Not_rain'] = X['Rainfall'].apply(lambda i : i == 0).astype('int')
    X['Not_snow'] = X['Snowfall'].apply(lambda i : i == 0).astype('int')
    X['Good_Temp'] = X['Temperature'].apply(lambda i : i >= 15 and i <=35).astype('int')
    X['Goop_Humid'] = X['Humidity'].apply(lambda i : i >= 20 and i <=80).astype('int')
    X['Good_Wind'] = X['Wind_Speed'].apply(lambda i : i >= 1.5 and i <=3.5).astype('int')
    return X['Not_rain'] + X['Not_snow'] + X['Good_Temp'] + X['Goop_Humid'] + X['Good_Wind']

data['Good_weather'] = weather_is_good(data)

sns.boxplot('Good_weather', 'Y', data=data)
plt.show()
data['Good_weather'].value_counts()

## Categorical features

In [ ]:
plot_subplots(cat_cols, lambda c : sns.boxplot(x=c, y='Y', data=data))

In [ ]:
def rush_hours(i):
    if i in [23,0,1,2,3,4,5,6, 7]:
        return 0
    elif i in [9,10,11,12,13,14,15,16]:
        return 1
    else:
        return 2

data['Rush_hours'] = data['Hour'].apply(rush_hours)
sns.boxplot('Rush_hours', 'Y', data=data)

In [ ]:
sns.pointplot('Month', 'Y', hue='Year', data=data)

In [ ]:
plt.figure(figsize=(20,8))
sns.pointplot('Hour', 'Y', hue='Dayofweek', data=data, ci=None)

## Target

In [ ]:
fig,axes = plt.subplots(2,2, figsize=(20, 10))
fig.suptitle('Target Distribution', fontsize=16, y=0.95)
sns.distplot(data["Y"],ax=axes[0][0])
sns.distplot(np.log1p(data["Y"]),ax=axes[0][1])
sns.boxplot(data['Y'], ax=axes[1][0])
sns.boxplot(np.log1p(data["Y"]), ax=axes[1][1])

# Feature engineering 

In [ ]:
def encode(df):
    df['Seasons'] = df['Seasons'].map({'Winter': 0, 'Spring': 1, 'Autumn': 2, 'Summer': 3})
    df['Functioning_Day'] = df['Functioning_Day'].map({'No': 0, 'Yes': 1})
    df['Holiday'] = df['Holiday'].map({'No Holiday': 0, 'Holiday': 1})
    df['Year'] = df['Year'].map({2017:0 , 2018: 1})
    
    return df

def add_time_features(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df.Date.dt.month
    df['Dayofweek'] = df.Date.dt.dayofweek
    df['Year'] = df.Date.dt.year
    df['Day'] = df.Date.dt.day
#     df['Rush_hours'] = df['Hour'].apply(rush_hours)
#     df['Good_weather'] = weather_is_good(df)
#     min_temp = df.groupby('Date')['Temperature'].min()
#     df = df.merge(min_temp, how='left', suffixes=('','_min'), on='Date')
#     max_temp = df.groupby('Date')['Temperature'].max()
#     df = df.merge(max_temp, how='left', suffixes=('','_max'), on='Date')
    return df

def lag_features(df):
    for feature in ['Temperature', 'Humidity', 'Solar_Radiation', 'Rainfall', 'Wind_Speed', 'Visibility']:
        for hours in [1,2,3,-1]:
            i = str(hours)
            df[feature + '_lag_' + i] = df[feature].shift(-hours)
            df[feature + '_lag_' + i] = np.where(df[feature + '_lag_' + i].isnull(),
                                                 df[feature],
                                                 df[feature + '_lag_' + i])
    return df

def select_features(df):
    return df.drop(['Id', 'Date', 'Snowfall'], axis=1)
    
def pipeline(df):
    df = add_time_features(df)
    df = encode(df)
    df = lag_features(df)
    df = select_features(df)
    return df

In [ ]:
X = df.drop('Y',axis=1)
y = df['Y']

X['Date'] = pd.to_datetime(X['Date'])
X['Day'] = X.Date.dt.day

future_days = [17,18,19,20]
X_train = X[~X.Date.dt.day.isin(future_days)]
y_train = y[~X.Date.dt.day.isin(future_days)]
X_val = X[X.Date.dt.day.isin(future_days)]
y_val = y[X.Date.dt.day.isin(future_days)]

X = X.drop(columns=['Day'])
X_train = X_train.drop(columns=['Day'])
X_val = X_val.drop(columns=['Day'])


X = pipeline(X)

X_train = pipeline(X_train)
X_val = pipeline(X_val)

In [ ]:
mi_scores = make_mi_scores(X, y, discrete_features=False)
plt.figure(figsize=(10,10))
plot_mi_scores(mi_scores)

# Model Building

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, (np.maximum(y_pred, 0))))

In [ ]:
cat = CatBoostRegressor(n_estimators= 450, max_depth= 5, learning_rate= 0.1 , verbose=0)

rfe_cat = RFE(cat, n_features_to_select=27)
model = rfe_cat.fit(X_train, np.log1p(y_train))

print("Train Score : ", rmsle(y_train, np.expm1(model.predict(X_train))))
pred = np.expm1(np.maximum(model.predict(X_val), 0))
print("Validation Score : ", rmsle(y_val, pred))

In [ ]:
xgb = XGBRegressor(n_estimators= 200, max_depth= 5, learning_rate= 0.1)

rfe_xgb = RFE(xgb, n_features_to_select=25)
model = rfe_xgb.fit(X_train, np.log1p(y_train))

print("Train Score : ", rmsle(y_train, np.expm1(model.predict(X_train))))
pred = np.expm1(np.maximum(model.predict(X_val), 0))
print("Validation Score : ", rmsle(y_val, pred))

In [ ]:
model = VotingRegressor([('xgb',rfe_xgb), ('cat', rfe_cat)], weights=[0.2, 0.8])

model.fit(X_train,np.log1p(y_train))

print("Train Score : ", rmsle(y_train, np.expm1(model.predict(X_train))))
pred = np.expm1(np.maximum(model.predict(X_val), 0))
print("Validation Score : ", rmsle(y_val, pred))

# Evaluation

In [ ]:
print("MAE Score : ", mean_absolute_error(y_val, pred))
print("RMSE Score : ", np.sqrt(mean_squared_error(y_val, pred)))
print("R2 Score : ", r2_score(y_val, pred))

In [ ]:
f, axs = plt.subplots(2,1, figsize=(10, 10))
axs[0].axvline(color='r')
axs[1].axvline(color='r')
sns.distplot(y_val - pred, bins=30, ax=axs[0])
sns.boxplot(y_val - pred, ax=axs[1])

In [ ]:
plt.scatter(y_val, pred, alpha=0.2)
plt.plot(y_val, y_val, color='r')
plt.show()

# Train All Data

In [ ]:
model.fit(X, np.log1p(y))
print("Final Score : ", rmsle(y, np.expm1(model.predict(X))))

# Submission File Generation

In [ ]:
df_test = pd.read_csv('../input/seoul-bike-rental-ai-pro-iti/test.csv')
df_test.columns = [i.split("(")[0].strip().title().replace(" ","_") for i in [*df_test.columns]]

pred = model.predict(pipeline(df_test))

submission = pd.read_csv('../input/seoul-bike-rental-ai-pro-iti/sample_submission.csv')
submission['y'] = np.maximum(np.expm1(pred),0)
submission.to_csv('my_submission.csv', index=False)